## Expected Features

References: 
- https://www.samford.edu/sports-analytics/fans/2023/How-I-Built-a-Competitive-NFL-Prediction-Model-with-Only-Five-Statistics
- https://github.com/theedgepredictor/nfl-feature-store
- https://github.com/theedgepredictor/nfl-model-store


### Raw Features

From our feature store. All features are rolled up for the current season up to the given week.

Features that are only available for offense OR defense OR classifying or target metrics
- Targets: Classification or Regression metrics for the given week (only available for completed games)
- Meta: Classifying stats for grouping team based stats
- Vegas: Latest Vegas lines (Historically mid - close week lines)
- ELO: ELO ratings 

Features that are available for both offense and defense:
- EWMA: Expected weighted moving averages
- Point: Point avgs and point differential avgs (whole game and quarters)
- Cover: Cover avgs (Last 10 games)
- Down: Down avgs 
- Fantasy: Fantasy points avg for whole team (Collected from offensive player stats and averaged)
- Common: Common features that are available across positions 
- Common Passing: Common passing features
- Common Rushing: Common rushing features
- Ranking: Ranking features (These might not be important for expected features but are available for both offense and defense)
- Kicking: Kickoff metrics


### 1. Config

In [1]:
import pandas as pd
import numpy as np
import datetime
from pandas.core.dtypes.common import is_numeric_dtype

from nfl_data_loader.api.feature_stores.events.events import get_event_feature_store
from nfl_data_loader.utils.formatters.general import df_rename_shift, df_rename_fold, df_rename_exavg
from nfl_data_loader.schemas.events.features import TARGETS, VEGAS, META, POINT_FEATURES, JUST_SIMPLE_FEATURES

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


### 2. PoC

Prove it out with Points Feature Group and then add in more features

In [5]:
#### Load data and split features into shifted and base
seasons = list(range(2023, 2026))
event_fs = pd.concat([get_event_feature_store(season) for season in seasons], ignore_index=True)
columns_for_base = META + ['home_elo_pre', 'away_elo_pre'] + VEGAS + TARGETS
columns_for_shift = ['team', 'season', 'week','is_home'] + POINT_FEATURES + JUST_SIMPLE_FEATURES
shifted_df = event_fs.copy()
base_dataset_df = event_fs[columns_for_base].copy()
del event_fs

#### Shift Features
shifted_df = df_rename_shift(shifted_df)[columns_for_shift]

#### Rename for Expected Average
t1_cols = [i for i in shifted_df.columns if '_offense' in i and (i not in TARGETS + META) and i.replace('home_', '') in columns_for_shift]
t2_cols = [i for i in shifted_df.columns if '_defense' in i and (i not in TARGETS + META) and i.replace('away_', '') in columns_for_shift]

#### Apply Expected Average
expected_features_df = df_rename_exavg(shifted_df, '_offense', '_defense', t1_cols=t1_cols, t2_cols=t2_cols)

#### Rename back into home and away features
home_exavg_features_df = expected_features_df[expected_features_df['is_home'] == 1].copy().drop(columns='is_home')
away_exavg_features_df = expected_features_df[expected_features_df['is_home'] == 0].copy().drop(columns='is_home')
home_exavg_features_df.columns = ["home_"+col if 'exavg_' in col or col == 'team' else col for col in home_exavg_features_df.columns]
away_exavg_features_df.columns = ["away_"+col if 'exavg_' in col or col == 'team' else col for col in away_exavg_features_df.columns]

#### Merge home and away Expected Average features into base as dataset_df
dataset_df = pd.merge(base_dataset_df, home_exavg_features_df, on=['home_team', 'season', 'week'], how='left')
dataset_df = pd.merge(dataset_df, away_exavg_features_df, on=['away_team', 'season', 'week'], how='left')

dataset_df['game_id'] = dataset_df.apply(lambda x: f"{x['season']}_{x['week']}_{x['home_team']}_{x['away_team']}", axis=1)

#### Fold base from away and home into team
folded_dataset_df = base_dataset_df.copy()
folded_dataset_df['game_id'] = folded_dataset_df.apply(lambda x: f"{x['season']}_{x['week']}_{x['home_team']}_{x['away_team']}", axis=1)
folded_dataset_df = folded_dataset_df.rename(columns={'spread_line': 'away_spread_line'})
folded_dataset_df['home_spread_line'] = - folded_dataset_df['away_spread_line']
folded_dataset_df['actual_home_spread'] = -folded_dataset_df['actual_away_spread']
folded_dataset_df['actual_home_team_win'] = folded_dataset_df['actual_away_team_win'] == 0
folded_dataset_df['actual_home_team_covered_spread'] = folded_dataset_df['actual_away_team_covered_spread'] == 0
folded_dataset_df = df_rename_fold(folded_dataset_df, 'away_', 'home_')
folded_dataset_df = pd.merge(folded_dataset_df, expected_features_df, on=['team', 'season', 'week'], how='left')
folded_dataset_df

,season,week,team,elo_pre,spread_line,total_line,actual_score,actual_team_win,actual_spread,actual_point_total,actual_team_covered_spread,actual_under_covered,game_id,is_home,exavg_avg_points,exavg_avg_point_differential,exavg_avg_q1_point_diff,exavg_avg_q2_point_diff,exavg_avg_q3_point_diff,exavg_avg_q4_point_diff,exavg_avg_q5_point_diff,exavg_avg_q1_points,exavg_avg_q2_points,exavg_avg_q3_points,exavg_avg_q4_points,exavg_avg_q5_points,exavg_avg_carries,exavg_avg_rushing_yards,exavg_avg_rushing_tds,exavg_avg_completions,exavg_avg_attempts,exavg_avg_passing_yards,exavg_avg_passing_tds,exavg_avg_time_of_possession,exavg_avg_turnover,exavg_avg_field_goal_made
0,2023,1,DET,1543.620986,4.0,53.0,21.0,True,-1.0,41.0,True,True,2023_1_KC_DET,0,24.527778,-2.722222,-0.055556,-0.305556,-3.000000,0.888889,-0.250000,4.944444,7.583333,4.361111,7.638889,0.000000,26.472222,119.222222,1.027778,22.944444,35.111111,252.194444,1.805556,1799.111111,1.138889,1.416667
1,2023,1,CAR,1459.403465,3.5,40.5,10.0,False,14.0,34.0,False,True,2023_1_ATL_CAR,0,21.666667,-0.305556,1.055556,1.916667,-1.888889,-1.388889,0.000000,4.138889,7.972222,3.472222,6.083333,0.000000,29.305556,131.361111,0.861111,19.000000,30.138889,214.805556,1.305556,1795.611111,1.111111,1.944444
2,2023,1,HOU,1351.067207,9.5,43.5,9.0,False,16.0,34.0,False,True,2023_1_BAL_HOU,0,17.916667,-4.444444,-1.416667,-2.472222,-0.833333,0.361111,-0.083333,2.861111,4.750000,4.055556,6.166667,0.083333,23.750000,88.472222,0.500000,22.194444,34.694444,234.416667,1.222222,1739.388889,1.500000,1.777778
3,2023,1,CIN,1661.261077,-1.0,46.5,3.0,False,21.0,27.0,False,True,2023_1_CLE_CIN,0,24.027778,3.361111,-0.222222,1.444444,0.861111,1.361111,-0.083333,3.888889,8.055556,5.333333,6.750000,0.000000,26.166667,112.250000,1.000000,22.027778,34.083333,244.027778,1.694444,1810.972222,1.027778,1.611111
4,2023,1,JAX,1544.674275,-4.0,45.5,31.0,True,-10.0,52.0,True,False,2023_1_IND_JAX,0,24.527778,6.166667,0.194444,2.694444,1.666667,1.444444,0.166667,3.972222,8.083333,5.444444,6.777778,0.250000,28.361111,122.972222,1.000000,22.361111,33.250000,234.166667,1.500000,1833.305556,1.166667,1.777778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1601,2025,17,SF,1438.923281,-3.5,46.5,NaN,True,NaN,NaN,True,False,2025_17_SF_CHI,1,22.058824,1.323529,3.029412,1.941176,-0.794118,-2.852941,0.000000,5.264706,7.264706,4.558824,4.882353,0.088235,27.558824,129.647059,1.058824,19.735294,30.176471,245.794118,1.205882,1819.264706,1.411765,2.000000
1602,2025,17,ATL,1431.200309,3.0,46.5,NaN,True,NaN,NaN,True,False,2025_17_ATL_LAR,1,21.882353,-1.176471,2.264706,-0.794118,-0.205882,-2.617647,0.176471,5.352941,6.617647,5.205882,4.352941,0.352941,28.352941,125.852941,0.882353,21.117647,32.235294,245.764706,1.382353,1818.352941,1.382353,1.705882
1603,2025,18,JAX,1396.470015,-4.5,43.5,NaN,True,NaN,NaN,True,False,2025_18_JAX_TEN,1,22.764706,0.617647,-0.941176,1.235294,-0.088235,0.500000,-0.088235,4.088235,7.147059,5.205882,6.323529,0.000000,26.470588,113.235294,0.941176,19.911765,30.735294,208.529412,1.411765,1705.058824,1.294118,1.558824
1604,2025,18,LAR,1562.374458,-4.5,45.5,NaN,True,NaN,NaN,True,False,2025_18_LAR_ARI,1,21.735294,-0.029412,-1.970588,1.705882,-0.823529,0.794118,0.264706,3.176471,7.235294,4.764706,6.205882,0.352941,27.264706,115.852941,0.970588,21.000000,31.352941,227.000000,1.235294,1791.970588,0.823529,1.764706


In [6]:
folded_dataset_df[((folded_dataset_df.season==2024)&(folded_dataset_df.week==1))].copy()

,season,week,team,elo_pre,spread_line,total_line,actual_score,actual_team_win,actual_spread,actual_point_total,actual_team_covered_spread,actual_under_covered,game_id,is_home,exavg_avg_points,exavg_avg_point_differential,exavg_avg_q1_point_diff,exavg_avg_q2_point_diff,exavg_avg_q3_point_diff,exavg_avg_q4_point_diff,exavg_avg_q5_point_diff,exavg_avg_q1_points,exavg_avg_q2_points,exavg_avg_q3_points,exavg_avg_q4_points,exavg_avg_q5_points,exavg_avg_carries,exavg_avg_rushing_yards,exavg_avg_rushing_tds,exavg_avg_completions,exavg_avg_attempts,exavg_avg_passing_yards,exavg_avg_passing_tds,exavg_avg_time_of_possession,exavg_avg_turnover,exavg_avg_field_goal_made
272,2024,1,BAL,1649.201559,3.0,46.0,20.0,False,7.0,47.0,False,False,2024_1_KC_BAL,0,22.388889,2.694444,0.583333,0.805556,0.805556,0.416667,0.083333,4.527778,7.666667,4.527778,5.500000,0.166667,28.472222,133.194444,1.027778,19.694444,31.388889,214.944444,1.305556,1825.888889,1.305556,1.638889
273,2024,1,GB,1569.961024,2.0,49.5,29.0,False,5.0,63.0,False,False,2024_1_PHI_GB,0,23.416667,0.833333,-0.944444,1.611111,-0.500000,0.583333,0.083333,3.916667,7.027778,5.638889,6.750000,0.083333,25.000000,108.194444,0.666667,23.305556,36.111111,253.722222,1.916667,1783.583333,1.000000,1.500000
274,2024,1,PIT,1540.475591,4.0,43.0,18.0,True,-8.0,28.0,True,True,2024_1_ATL_PIT,0,20.055556,1.000000,-0.222222,0.722222,0.750000,-0.250000,0.000000,3.694444,6.555556,4.666667,5.138889,0.000000,28.972222,118.027778,0.805556,19.388889,31.055556,207.194444,1.111111,1807.416667,0.916667,1.972222
275,2024,1,ARI,1426.831139,6.5,46.0,28.0,False,6.0,62.0,True,False,2024_1_BUF_ARI,0,18.805556,-8.388889,-0.888889,-2.750000,-2.472222,-2.527778,0.250000,4.194444,6.000000,2.944444,5.333333,0.333333,25.638889,122.583333,0.888889,21.222222,32.611111,209.944444,1.111111,1733.583333,1.611111,1.527778
276,2024,1,TEN,1430.781514,4.0,43.0,17.0,False,7.0,41.0,False,True,2024_1_CHI_TEN,0,20.250000,-0.861111,-0.305556,1.444444,-1.083333,-0.750000,-0.166667,3.388889,7.527778,4.000000,5.166667,0.166667,25.250000,100.055556,0.722222,21.166667,32.583333,229.916667,1.305556,1761.972222,1.388889,1.638889
277,2024,1,NE,1438.668492,8.0,40.5,16.0,True,-6.0,26.0,True,True,2024_1_CIN_NE,0,18.305556,-3.666667,-2.083333,-0.111111,-0.500000,-0.888889,-0.083333,3.222222,6.277778,3.888889,4.916667,0.000000,25.583333,110.805556,0.750000,21.111111,33.222222,233.361111,1.166667,1745.083333,1.750000,1.444444
278,2024,1,HOU,1515.255614,-3.0,49.0,29.0,True,-2.0,56.0,False,False,2024_1_IND_HOU,0,23.777778,1.416667,0.138889,1.527778,-1.416667,1.083333,0.083333,5.166667,6.972222,4.611111,6.694444,0.333333,28.000000,107.222222,0.888889,21.944444,34.444444,259.916667,1.500000,1829.527778,1.333333,2.000000
279,2024,1,JAX,1503.872530,3.5,49.5,17.0,False,3.0,37.0,True,True,2024_1_MIA_JAX,0,22.055556,-2.694444,-0.111111,-1.222222,-2.000000,0.722222,-0.083333,4.000000,6.777778,4.388889,6.888889,0.000000,25.555556,93.138889,0.888889,23.055556,34.805556,246.388889,1.388889,1801.694444,1.611111,1.444444
280,2024,1,CAR,1335.672079,3.5,41.5,10.0,False,37.0,57.0,False,False,2024_1_NO_CAR,0,16.194444,-6.916667,-0.361111,-2.527778,-2.027778,-2.000000,0.000000,3.583333,4.833333,2.722222,5.055556,0.000000,27.472222,115.250000,0.472222,19.250000,32.611111,197.194444,0.972222,1801.361111,1.500000,1.583333
281,2024,1,MIN,1476.142009,-1.0,42.5,28.0,True,-22.0,34.0,True,True,2024_1_NYG_MIN,0,22.277778,4.194444,1.500000,2.611111,0.138889,-0.138889,0.083333,4.111111,7.194444,5.694444,5.194444,0.083333,26.111111,113.333333,0.944444,23.416667,35.583333,261.861111,1.444444,1811.527778,1.916667,1.583333


### 3. Example Event

In [10]:
home_team = 'NYG'
away_team = 'MIN'
season = 2024
week = 1

matchup = dataset_df[(
    (dataset_df['home_team'] == home_team) & (dataset_df['away_team'] == away_team) & (dataset_df['season'] == season) & (dataset_df['week'] == week)
)].copy()

home = folded_dataset_df[(
    (folded_dataset_df['team'] == home_team) & (folded_dataset_df['season'] == season) & (folded_dataset_df['week'] == week)
)]

away = folded_dataset_df[(
    (folded_dataset_df['team'] == away_team) & (folded_dataset_df['season'] == season) & (folded_dataset_df['week'] == week)
)]

## Note: looks like all point diffs need to be flipped to be correct
folded_matchup = pd.concat([home.T, away.T], axis=1)

print(f"Vegas lines for {home_team} vs. {away_team} in Week {week} of Season {season}:")
print(f"-- Spread: {away['spread_line'].values[0].round(2)} (towards away team)")
print(f"-- Total: {away['total_line'].values[0].round(2)}")
print()

print(f"Home Team Stats: {home_team}")
print(f"-- Rating: {home['elo_pre'].values[0].round(2)}")
print(f"-- Expected Score Q1 ")
print(f"-- Expected Score: {home['exavg_avg_points'].values[0].round(2)}")
print()

print(f"Away Team Stats: {away_team}")
print(f"-- Rating: {away['elo_pre'].values[0].round(2)}")
print(f"-- Expected Score: {away['exavg_avg_points'].values[0].round(2)}")
print()

print(f"Result")
print(f"-- Spread: Expected {home['exavg_avg_point_differential'].values[0].round(2)} (Actual {away['actual_spread'].values[0].round(2)})")
print(f"-- Total: Expected {home['exavg_avg_points'].values[0].round(2) + away['exavg_avg_points'].values[0].round(2)} (Actual {away['actual_point_total'].values[0].round(2)})")
folded_matchup

Vegas lines for NYG vs. MIN in Week 1 of Season 2024:
-- Spread: -1.0 (towards away team)
-- Total: 42.5

Home Team Stats: NYG
-- Rating: 1457.43
-- Expected Score Q1 
-- Expected Score: 18.25

Away Team Stats: MIN
-- Rating: 1476.14
-- Expected Score: 22.28

Result
-- Spread: Expected -4.03 (Actual -22.0)
-- Total: Expected 40.53 (Actual 34.0)


,1084,281
season,2024,2024
week,1,1
team,NYG,MIN
elo_pre,1457.432535,1476.142009
spread_line,1.0,-1.0
total_line,42.5,42.5
actual_score,6.0,28.0
actual_team_win,False,True
actual_spread,22.0,-22.0
actual_point_total,34.0,34.0


In [18]:
matchup.columns

Index(['season', 'week', 'home_team', 'away_team', 'home_elo_pre',
       'away_elo_pre', 'spread_line', 'total_line', 'actual_home_score',
       'actual_away_score', 'actual_away_team_win', 'actual_away_spread',
       'actual_point_total', 'actual_away_team_covered_spread',
       'actual_under_covered', 'home_exavg_avg_points',
       'home_exavg_avg_point_differential', 'home_exavg_avg_q1_point_diff',
       'home_exavg_avg_q2_point_diff', 'home_exavg_avg_q3_point_diff',
       'home_exavg_avg_q4_point_diff', 'home_exavg_avg_q5_point_diff',
       'home_exavg_avg_q1_points', 'home_exavg_avg_q2_points',
       'home_exavg_avg_q3_points', 'home_exavg_avg_q4_points',
       'home_exavg_avg_q5_points', 'home_exavg_avg_carries',
       'home_exavg_avg_rushing_yards', 'home_exavg_avg_rushing_tds',
       'home_exavg_avg_completions', 'home_exavg_avg_attempts',
       'home_exavg_avg_passing_yards', 'home_exavg_avg_passing_tds',
       'home_exavg_avg_time_of_possession', 'home_exavg_avg

In [11]:
matchup.T

,281
season,2024
week,1
home_team,NYG
away_team,MIN
home_elo_pre,1457.432535
away_elo_pre,1476.142009
spread_line,-1.0
total_line,42.5
actual_home_score,6.0
actual_away_score,28.0


### 4. Evaluation

In [22]:
from sklearn.metrics import accuracy_score, mean_absolute_error

eval_season = 2024
eval_df = dataset_df[((dataset_df['season'] == eval_season)) & (dataset_df['actual_away_score'].notnull())].copy()
eval_df['expected_spread'] = eval_df['home_exavg_avg_points'] - eval_df['away_exavg_avg_points']
eval_df['expected_total'] = eval_df['home_exavg_avg_points'] + eval_df['away_exavg_avg_points']

actual_wp = eval_df['actual_away_team_win'].values
actual_spread = eval_df['actual_away_spread'].values
actual_total = eval_df['actual_point_total'].values

print(f'Evaluation Report for the {eval_season} Season')
print('-- The away team won: ', round(sum(actual_wp) / len(actual_wp), 4) * 100, '% of the time')
print(f"-- The average score differential was {round(actual_spread.mean(), 2)} (abs: {round(np.abs(actual_spread).mean(), 2)})")
print(f"-- The average total was {round(actual_total.mean(), 2)} with a low of {round(actual_total.min(), 2)} and a high of {round(actual_total.max(), 2)}")

print()
print('Vegas Baseline Scores')
vegas_wp = eval_df['spread_line'].apply(lambda x: 1 if x < 0 else 0)
vegas_spread = eval_df['spread_line'].values
vegas_total = eval_df['total_line'].values

print(f"-- Vegas WP: {accuracy_score(actual_wp, vegas_wp)}")
print(f"-- Vegas Spread: {mean_absolute_error(actual_spread, vegas_spread)}")
print(f"-- Vegas Total: {mean_absolute_error(actual_total, vegas_total)}")
print()

print('Expected Points Averages Scores')
exp_avg_wp = eval_df['home_exavg_avg_point_differential'].apply(lambda x: 1 if x < 0 else 0)
exp_avg_spread = eval_df['home_exavg_avg_point_differential'].values
exp_avg_total = eval_df['home_exavg_avg_points'].values + eval_df['away_exavg_avg_points'].values

print(f"-- Expected WP: {accuracy_score(actual_wp, exp_avg_wp)}")
print(f"-- Expected Spread: {mean_absolute_error(actual_spread, exp_avg_spread)}")
print(f"-- Expected Total: {mean_absolute_error(actual_total, exp_avg_total)}")

eval_df['expected_system_covered_spread'] = (eval_df['away_exavg_avg_points'] + eval_df['spread_line'] >= eval_df['home_exavg_avg_points'])
eval_df['expected_system_covered_spread'] = eval_df['expected_system_covered_spread'] == eval_df['actual_away_team_covered_spread']

# Calculate if the game covered the under
eval_df['expected_system_under_covered_total'] = (eval_df['home_exavg_avg_points'] + eval_df['away_exavg_avg_points'] <= eval_df['total_line'])

eval_df.expected_system_covered_spread.sum() / len(eval_df)

Evaluation Report for the 2024 Season
-- The away team won:  46.69 % of the time
-- The average score differential was 1.87 (abs: 11.2)
-- The average total was 45.82 with a low of 9.0 and a high of 90.0

Vegas Baseline Scores
-- Vegas WP: 0.7132352941176471
-- Vegas Spread: 9.610294117647058
-- Vegas Total: 9.729779411764707

Expected Points Averages Scores
-- Expected WP: 0.6397058823529411
-- Expected Spread: 10.372704084140342
-- Expected Total: 10.135888041456814


0.5404411764705882

In [35]:

report = eval_df[
    [
        'season', 
        'week', 
        'away_team', 
        'home_team', 
        'actual_away_spread', 
        'spread_line', 
        'expected_spread',
        'actual_away_team_covered_spread', 
        'expected_system_covered_spread',
        'total_line', 
        'actual_home_score', 
        'actual_away_score',
        'actual_away_team_win', 
        'actual_point_total',
        'actual_under_covered',
        'expected_total', 
        
        'expected_system_under_covered_total'
       ]]
report['unit'] = 1000

report.loc[report.expected_system_covered_spread == False,'unit'] = -1000

report.unit.sum() * (1-.0476) ## 4.76% is the vig on the bet, so we assume we lose that on every bet

C:\Users\charl\AppData\Local\Temp\ipykernel_35588\553622294.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report['unit'] = 1000


20952.8

In [10]:
eval_df.head()

,season,week,home_team,away_team,home_elo_pre,away_elo_pre,spread_line,total_line,home_score,away_score,away_team_win,away_team_spread,total_target,away_team_covered,home_team_covered,under_covered,away_team_covered_spread,home_exavg_avg_points,home_exavg_avg_point_differential,home_exavg_avg_q1_point_diff,home_exavg_avg_q2_point_diff,home_exavg_avg_q3_point_diff,home_exavg_avg_q4_point_diff,home_exavg_avg_q5_point_diff,home_exavg_avg_q1_points,home_exavg_avg_q2_points,home_exavg_avg_q3_points,home_exavg_avg_q4_points,home_exavg_avg_q5_points,home_exavg_avg_carries,home_exavg_avg_rushing_yards,home_exavg_avg_rushing_tds,home_exavg_avg_completions,home_exavg_avg_attempts,home_exavg_avg_passing_yards,home_exavg_avg_passing_tds,home_exavg_avg_time_of_possession,home_exavg_avg_turnover,home_exavg_avg_field_goal_made,away_exavg_avg_points,away_exavg_avg_point_differential,away_exavg_avg_q1_point_diff,away_exavg_avg_q2_point_diff,away_exavg_avg_q3_point_diff,away_exavg_avg_q4_point_diff,away_exavg_avg_q5_point_diff,away_exavg_avg_q1_points,away_exavg_avg_q2_points,away_exavg_avg_q3_points,away_exavg_avg_q4_points,away_exavg_avg_q5_points,away_exavg_avg_carries,away_exavg_avg_rushing_yards,away_exavg_avg_rushing_tds,away_exavg_avg_completions,away_exavg_avg_attempts,away_exavg_avg_passing_yards,away_exavg_avg_passing_tds,away_exavg_avg_time_of_possession,away_exavg_avg_turnover,away_exavg_avg_field_goal_made,game_id,expected_spread,expected_total,expected_covered_spread,vegas_covered_spread,expected_covered_total,vegas_covered_total
272,2024,1,KC,BAL,1706.761563,1681.822751,3.0,46.0,27.0,20.0,0,7.0,47.0,0,1,0,0,19.694444,-2.694444,-0.583333,0.194444,-1.805556,-0.416667,-0.083333,3.944444,7.444444,3.138889,5.083333,0.083333,24.444444,107.416667,0.527778,23.555556,37.138889,236.583333,1.333333,1792.694444,1.527778,1.888889,22.388889,2.694444,0.583333,0.805556,0.805556,0.416667,0.083333,4.527778,7.666667,4.527778,5.500000,0.166667,28.472222,133.194444,1.027778,19.694444,31.388889,214.944444,1.305556,1825.888889,1.305556,1.638889,2024_1_KC_BAL,-2.694444,42.083333,True,False,False,True
273,2024,1,PHI,GB,1544.887369,1579.881723,2.0,49.5,34.0,29.0,0,5.0,63.0,0,1,0,0,22.916667,-0.500000,0.944444,-0.666667,-0.444444,-0.305556,-0.027778,4.861111,6.027778,5.527778,6.250000,0.250000,29.500000,128.055556,1.111111,20.944444,32.083333,231.194444,1.250000,1838.666667,1.361111,1.888889,23.416667,0.833333,-0.944444,1.611111,-0.500000,0.583333,0.083333,3.916667,7.027778,5.638889,6.750000,0.083333,25.000000,108.194444,0.666667,23.305556,36.111111,253.722222,1.916667,1783.583333,1.000000,1.500000,2024_1_PHI_GB,-0.500000,46.333333,True,False,False,True
274,2024,1,ATL,PIT,1406.881419,1536.797127,4.0,43.0,10.0,18.0,1,-8.0,28.0,1,0,1,1,19.138889,-0.750000,0.222222,-0.305556,-1.166667,0.500000,0.000000,3.916667,6.166667,3.583333,5.472222,0.000000,28.694444,122.861111,0.666667,19.805556,32.277778,233.138889,1.222222,1792.583333,1.638889,1.861111,20.055556,1.000000,-0.222222,0.722222,0.750000,-0.250000,0.000000,3.694444,6.555556,4.666667,5.138889,0.000000,28.972222,118.027778,0.805556,19.388889,31.055556,207.194444,1.111111,1807.416667,0.916667,1.972222,2024_1_ATL_PIT,-0.916667,39.194444,True,True,False,False
275,2024,1,BUF,ARI,1679.473922,1377.288327,6.5,46.0,34.0,28.0,0,6.0,62.0,1,0,0,1,27.194444,8.638889,0.888889,3.166667,2.055556,2.611111,-0.083333,5.083333,9.000000,5.166667,7.861111,0.083333,30.416667,136.250000,1.194444,21.277778,31.555556,238.444444,1.861111,1880.333333,1.250000,1.416667,18.805556,-8.388889,-0.888889,-2.750000,-2.472222,-2.527778,0.250000,4.194444,6.000000,2.944444,5.333333,0.333333,25.638889,122.583333,0.888889,21.222222,32.611111,209.944444,1.111111,1733.583333,1.611111,1.527778,2024_1_BUF_ARI,8.388889,46.000000,False,True,False,True
276,2024,1,CHI,TEN,1439.277180,1413.184182,4.0,43.0,24.0,17.0,0,7.0,41.0,0,1,1,0,21.111111,0.944444,0.305556,-0.277778,-0.083333,0.833333,0.166667,3.694444,6.416667,4.750000,6.000000,0.250000,29.166667,121

In [32]:
s = folded_dataset_df[folded_dataset_df['season'] == 2024].copy()
s['points_over_expected'] = s['actual_score'] - s['exavg_avg_points']
s.groupby(['team'])['points_over_expected'].mean().sort_values(ascending=False).reset_index()

,team,points_over_expected
0,DET,7.729295
1,BAL,5.903687
2,BUF,4.893359
3,TB,4.477222
4,DEN,4.069567
5,CIN,3.957477
6,PHI,3.618798
7,WAS,3.523621
8,GB,3.173722
9,LAC,1.934469


In [33]:
dataset_df[((dataset_df['season'] == 2025)&(dataset_df['week'] == 1))].copy()

,season,week,home_team,away_team,home_elo_pre,away_elo_pre,spread_line,total_line,actual_home_score,actual_away_score,actual_away_team_win,actual_away_spread,actual_point_total,actual_away_team_covered_spread,actual_under_covered,home_exavg_avg_points,home_exavg_avg_point_differential,home_exavg_avg_q1_point_diff,home_exavg_avg_q2_point_diff,home_exavg_avg_q3_point_diff,home_exavg_avg_q4_point_diff,home_exavg_avg_q5_point_diff,home_exavg_avg_q1_points,home_exavg_avg_q2_points,home_exavg_avg_q3_points,home_exavg_avg_q4_points,home_exavg_avg_q5_points,home_exavg_avg_carries,home_exavg_avg_rushing_yards,home_exavg_avg_rushing_tds,home_exavg_avg_completions,home_exavg_avg_attempts,home_exavg_avg_passing_yards,home_exavg_avg_passing_tds,home_exavg_avg_time_of_possession,home_exavg_avg_turnover,home_exavg_avg_field_goal_made,away_exavg_avg_points,away_exavg_avg_point_differential,away_exavg_avg_q1_point_diff,away_exavg_avg_q2_point_diff,away_exavg_avg_q3_point_diff,away_exavg_avg_q4_point_diff,away_exavg_avg_q5_point_diff,away_exavg_avg_q1_points,away_exavg_avg_q2_points,away_exavg_avg_q3_points,away_exavg_avg_q4_points,away_exavg_avg_q5_points,away_exavg_avg_carries,away_exavg_avg_rushing_yards,away_exavg_avg_rushing_tds,away_exavg_avg_completions,away_exavg_avg_attempts,away_exavg_avg_passing_yards,away_exavg_avg_passing_tds,away_exavg_avg_time_of_possession,away_exavg_avg_turnover,away_exavg_avg_field_goal_made,game_id
544,2025,1,PHI,DAL,1690.600727,1450.271102,7.0,46.5,NaN,NaN,False,NaN,NaN,False,False,26.705882,6.529412,0.264706,1.941176,1.911765,2.411765,0.000000,4.529412,8.264706,6.264706,7.647059,0.000000,32.382353,158.764706,1.558824,18.941176,28.000000,219.235294,1.470588,1893.911765,1.264706,1.647059,20.176471,-6.529412,-0.264706,-1.264706,-2.588235,-2.411765,0.000000,4.264706,6.500000,4.176471,5.235294,0.000000,24.705882,101.941176,0.500000,22.117647,35.000000,223.411765,1.441176,1706.088235,1.617647,1.823529,2025_1_PHI_DAL
545,2025,1,LAC,KC,1550.250130,1646.031647,-3.0,45.5,NaN,NaN,False,NaN,NaN,False,False,20.000000,0.176471,0.176471,0.000000,0.823529,-0.823529,0.000000,3.911765,6.705882,4.264706,5.117647,0.000000,25.382353,104.529412,0.823529,20.970588,32.588235,227.058824,1.205882,1770.794118,0.852941,1.941176,20.352941,0.352941,-0.176471,0.794118,-1.617647,1.176471,0.176471,3.735294,6.705882,3.441176,6.294118,0.176471,26.735294,116.676471,0.647059,22.529412,34.470588,229.205882,1.411765,1839.558824,1.117647,1.823529,2025_1_LAC_KC
546,2025,1,ATL,TB,1431.200309,1569.036406,-2.5,48.5,NaN,NaN,False,NaN,NaN,False,False,21.588235,-5.029412,-0.823529,0.058824,-0.852941,-3.764706,0.352941,4.647059,8.382353,4.352941,3.676471,0.529412,25.911765,111.764706,0.823529,22.676471,34.147059,253.147059,1.382353,1740.970588,1.323529,1.705882,26.794118,5.911765,0.823529,0.294118,0.500000,4.029412,0.264706,5.470588,8.500000,5.029412,7.529412,0.264706,27.764706,132.941176,0.735294,23.735294,33.588235,247.617647,2.176471,1899.264706,1.205882,1.852941,2025_1_ATL_TB
547,2025,1,CLE,CIN,1402.162948,1585.173551,-5.5,44.5,NaN,NaN,False,NaN,NaN,False,False,20.382353,-6.352941,-2.205882,-3.147059,-2.617647,1.529412,0.088235,3.294118,5.058824,4.264706,7.676471,0.088235,25.764706,112.000000,0.794118,22.294118,35.911765,232.235294,1.500000,1798.764706,1.705882,1.117647,26.911765,6.794118,2.205882,3.676471,2.088235,-1.264706,0.088235,5.500000,8.470588,6.617647,6.147059,0.176471,25.647059,113.441176,0.970588,22.500000,33.558824,254.529412,2.117647,1828.294118,1.029412,1.323529,2025_1_CLE_CIN
548,2025,1,IND,MIA,1467.482962,1455.866640,1.5,45.5,NaN,NaN,False,NaN,NaN,False,False,21.264706,-1.000000,0.264706,0.117647,-0.852941,-0.529412,0.000000,4.058824,6.088235,4.088235,7.029412,0.000000,27.000000,121.794118,0.941176,19.147059,31.470588,217.352941,1.147059,1700.470588,1.441176,1.911765,22.529412,1.264706,-0.264706,0.764706,-0.029412,0.617647,0.176471,3.794118,6.235294,4.676471,7.647059,0.176471,28.529412,116.352941,0.882353,23.029412,32.823529,240.08823